# Plan and Execute for Deep Web Search 

- This code is designed to plan and execute a search on the deep web using a specified search engine.


In [4]:

import sys
import os
from openai import AzureOpenAI
import requests
import json
from urllib.parse import urljoin
from langchain_core.prompts import PromptTemplate
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from langchain_core.prompts import load_prompt
import sys
from utils.search_utils import web_search, url_search, extract_contexts_async
from IPython.display import Markdown, display
from datetime import datetime
import pytz
import time
import os
from dotenv import load_dotenv
load_dotenv(override=True) 
import logging

logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Get credentials from environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 10))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o-mini")

# Web search mode: "google" or "bing"
web_search_mode = "bing"


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

current_dir = os.getcwd()

def rewrite_query_for_search(query, client: AzureOpenAI):
    
  prompt_path = os.path.join(current_dir, "prompts", "rewrite_prompt.yaml")
  QUERY_REWRITE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
  response = client.chat.completions.create(
      model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
      messages=[
          {"role": "system", "content": QUERY_REWRITE_PROMPT.format(
            user_query=query)},
          {"role": "user", "content": query}
      ],
      temperature=0.8,
      max_tokens=300,
      response_format= {"type": "json_object"},
  )
  
  return json.loads(response.choices[0].message.content.strip())
    
  
def plan_query_for_search(query, client: AzureOpenAI):

  current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")
  
  prompt_path = os.path.join(current_dir, "prompts", "planner_prompt.yaml")
  PLANNER_PROMPT = load_prompt(prompt_path, encoding="utf-8")
  
  response = client.chat.completions.create(
      model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
      messages=[
          {"role": "system", "content": PLANNER_PROMPT.format(current_date=current_date, query=query)},
          {"role": "user", "content": query}
      ],
      temperature=0.8,
      max_tokens=300,
      response_format= {"type": "json_object"},
  )
  
  return json.loads(response.choices[0].message.content.strip())


In [5]:


   

# 웹 검색 결과를 활용해 LLM 답변을 생성하는 비동기 함수
async def process_deep_search(max_result_count=3, input_query=None, web_search_mode=None, product_name=None):
    
    start_time = time.time()
    print(f"Original Input: {input_query}")

    # 검색 모드가 지정되지 않으면 환경변수
    if web_search_mode is None:
        web_search_mode = os.getenv("WEB_SEARCH_MODE", "google").lower()
    print(f"############## Web Search Mode: {web_search_mode}")
    
    # query rewrite (검색용/LLM용)
    query_rewrite = rewrite_query_for_search(input_query, client)
    print(f"Web Search Query: {query_rewrite['search_query']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    # plan deep query
    search_plan_json = plan_query_for_search(query_rewrite['search_query'], client)
    print(f"Planned Search Queries: {search_plan_json}")
    search_queries = search_plan_json.get("search_queries", [])
    all_contexts = []
    
    for i, query in enumerate(search_queries):
        print(f"Processing query {i+1}: {query}")
        search_results = url_search(query, max_result_count, web_search_mode=web_search_mode, product_name=product_name)
        
        print(f"Search Results: {len(search_results)} results found for query '{query}'")
        
        print("Analyze search results...")        
        if search_results:
            url_snippet_tuples = [(r["link"], r["snippet"]) for r in search_results]
            contexts = await extract_contexts_async(url_snippet_tuples=url_snippet_tuples)
            
            formatted_contexts = [
                f"[search keyword: {query}]\n{context}"
                for context in contexts
            ]
            
            all_contexts.extend(formatted_contexts)
        
    
    
    current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")
        
    # Get the current working directory for this notebook
    prompt_path = os.path.join(current_dir, "prompts", "generate_prompt.yaml")
    GENERATE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
    
    print("Generate response...")            
    
    answer_messages = [
        {"role": "system", "content": GENERATE_PROMPT.format(
            product_name=product_name,
            date=current_date,
            contexts=all_contexts,
            user_query=query_rewrite['llm_query'],
        )},
        {"role": "user", "content": query_rewrite['llm_query']}
    ]
    
    response = client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        messages=answer_messages,
        top_p=0.9,
        max_tokens=1500
    )
    display(Markdown(response.choices[0].message.content))
    
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds\n\n")


In [7]:
RESULTS_COUNT = 5

inputs = [
    "갤럭시 s25와 아이폰4 비교해줘"
]

web_search_mode = "bing"

for input in inputs:
    print(f"Bing Grounding 검색 사용: {input}")
    await process_deep_search(max_result_count=RESULTS_COUNT, input_query=input, web_search_mode=web_search_mode, product_name="삼성전자")  # product_name은 필요에 따라 변경 가능

Bing Grounding 검색 사용: 갤럭시 s25와 아이폰4 비교해줘
Original Input: 갤럭시 s25와 아이폰4 비교해줘
############## Web Search Mode: bing


Web Search Query: 갤럭시 S25 아이폰 4 비교
LLM Query: 갤럭시 S25와 아이폰 4의 주요 사양과 기능을 비교해 주세요. 두 모델의 성능, 카메라, 배터리 수명, 디자인 등에서의 차이점과 장단점을 상세히 설명해 주세요.
Planned Search Queries: {'search_queries': ['갤럭시 S25 아이폰 4 비교', '갤럭시 S25 사양', '아이폰 4 사양']}
Processing query 1: 갤럭시 S25 아이폰 4 비교
Search Results: 0 results found for query '갤럭시 S25 아이폰 4 비교'
Analyze search results...
Processing query 2: 갤럭시 S25 사양
Search Results: 3 results found for query '갤럭시 S25 사양'
Analyze search results...
Processing query 3: 아이폰 4 사양


2025-06-17 10:53:01,053 - utils.search_utils - ERROR - Request failed: Client error '403 Forbidden' for url 'https://namu.wiki/w/iPhone%204'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403


Search Results: 3 results found for query '아이폰 4 사양'
Analyze search results...
Generate response...


갤럭시 S25와 아이폰 4는 각각 2025년과 2010년에 출시된 스마트폰으로, 기술적 발전이 매우 크게 차이나는 두 모델입니다. 아래에서 성능, 카메라, 배터리 수명, 디자인 등의 차이점과 장단점을 상세히 비교해 보겠습니다.

### 1. 성능
| 특징            | 갤럭시 S25                          | 아이폰 4                         |
|-----------------|------------------------------------|----------------------------------|
| **프로세서**    | 퀄컴 스냅드래곤 8 3세대            | Apple A4 (싱글코어)             |
| **RAM**         | 12GB                               | 512MB                            |
| **운영 체제**   | Android (최신 버전)               | iOS 4                            |
| **멀티태스킹**  | 뛰어난 멀티코어 성능               | 지원 (iOS 4에서 처음 도입)      |

**장점 및 단점:**
- **갤럭시 S25**는 최신의 프로세서를 사용해 멀티태스킹과 고사양 게임에서 우수한 성능을 발휘합니다.
- **아이폰 4**는 출시 당시 혁신적인 성능을 제공했으나, 현재의 고사양 앱과 게임을 원활하게 실행하기에는 한계가 있습니다.

### 2. 카메라
| 특징                 | 갤럭시 S25                          | 아이폰 4                         |
|----------------------|------------------------------------|----------------------------------|
| **후면 카메라**      | 2억 화소, 10배 광학 줌            | 500만 화소                       |
| **전면 카메라**      | 고화질 (구체적 정보 없음)         | VGA급                            |
| **동영상 촬영**      | 8K 촬영 가능                       | 720p 동영상 촬영                |

**장점 및 단점:**
- **갤럭시 S25**는 고화소 카메라와 향상된 이미지 처리 기술로 다양한 환경에서 뛰어난 사진 품질을 제공합니다.
- **아이폰 4**는 당시의 카메라로는 훌륭했으나, 현재 기준으로 보면 화소와 기능에서 많이 부족합니다.

### 3. 배터리 수명
| 특징             | 갤럭시 S25                   | 아이폰 4                     |
|------------------|------------------------------|------------------------------|
| **배터리 용량**  | 5500mAh (Ultra 모델)         | 약 1420mAh                   |
| **사용 시간**    | 하루 이상 충분히 사용 가능   | 7시간 통화, 대기시간 300시간|

**장점 및 단점:**
- **갤럭시 S25**는 대용량 배터리로 높은 지속 시간을 자랑합니다.
- **아이폰 4**는 배터리 효율이 좋았으나, 용량이 작아 현대적인 사용 환경에서는 빈번한 충전이 필요할 수 있습니다.

### 4. 디자인
| 특징                 | 갤럭시 S25                            | 아이폰 4                       |
|----------------------|--------------------------------------|--------------------------------|
| **크기**             | 더 크고 무겁다 (구체적 수치 미제공) | 115.2 x 58.6 x 9.3 mm         |
| **무게**             | 약 200g (구체적 수치 미제공)        | 약 137g                        |
| **재질**             | 프리미엄 유리 및 금속               | 강화유리 및 스테인리스 스틸   |
| **디스플레이**       | 최신 AMOLED, 고해상도               | 3.5인치 레티나 디스플레이     |

**장점 및 단점:**
- **갤럭시 S25**는 현대적이고 세련된 디자인을 갖추고 있으며, 디스플레이 기술에서도 한층 진보했습니다.
- **아이폰 4**는 당시 기준으로 아름다운 디자인을 제공했으나, 현재의 스펙트럼에선 크기와 무게가 불리할 수 있습니다.

### 결론
갤럭시 S25는 최신 기술과 성능을 갖춘 플래그십 모델로, 다양한 기능과 우수한 카메라 성능을 제공합니다. 반면 아이폰 4는 스마트폰 역사에서 중요한 모델이지만, 현대 기준으로는 성능, 카메라, 배터리 등에서 많이 뒤처져 있습니다. 사용자의 필요에 따라, 최신 모델인 갤럭시 S25가 훨씬 더 매력적인 선택이 될 것입니다.

elapsed time: 61.57 seconds


